In [8]:
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold
import optuna.integration.lightgbm as lgb

In [2]:
all_new = pd.read_csv('data/feature_df.csv')
wordvec_df = pd.read_csv('sub_notebook/data/wordvectorized_df.csv')
bert_embeded = pd.read_csv('kaggle-notebook/data/bert_embeded.csv')
#tfidf_1000 = pd.read_csv('data/tfidf_1000.csv')
tfidf_svd_raw_64 = pd.read_csv("data/tfidf_svd_raw_64.csv")
#tfidf_svd_content_64 = pd.read_csv("data/tfidf_svd_content_64.csv")
#tfidf_svd_compiled_64 = pd.read_csv("data/tfidf_svd_compiled_64.csv")
mfw_df = pd.read_csv('data/mfw.csv')
miw_df = pd.read_csv('data/miw.csv')

In [3]:
all_new = pd.merge(all_new, bert_embeded, on="id", how="outer")
all_new = pd.merge(all_new, tfidf_svd_raw_64, on="id", how="outer")
all_new = pd.merge(all_new, wordvec_df, on="id", how="outer")
all_new = pd.merge(all_new, miw_df, on="id", how="outer")
all_new = pd.merge(all_new, mfw_df.drop(["mfw", "sfw"], axis=1), on="id", how="outer")

In [37]:
all_new["data_type"][all_new["data_type"] == "train"] = int(0)
all_new["data_type"][all_new["data_type"] == "test"] = int(1)
all_new["data_type"] = all_new["data_type"].astype(int)
all_new.head()

<ipython-input-37-40a25e62ff7a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_new["data_type"][all_new["data_type"] == "train"] = int(0)
<ipython-input-37-40a25e62ff7a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_new["data_type"][all_new["data_type"] == "test"] = int(1)


,id,country,duration,category1,category2,state,data_type,number_of_chars,number_of_words,number_of_sentences,...,wordvec_15_y,wordvec_16_y,wordvec_17_y,wordvec_18_y,wordvec_19_y,wordvec_20_y,wordvec_21_y,wordvec_22_y,wordvec_23_y,wordvec_24_y
0,0,4,29,12,143,0.0,0,5289,961,40,...,0.10287,-0.17618,-1.28810,-0.598010,0.26131,-1.26190,0.392020,0.59309,-0.55232,0.005087
1,1,16,34,5,108,0.0,0,1144,202,9,...,0.35207,0.16624,-0.91364,0.693300,-0.65035,0.77353,-0.087782,-0.43228,-0.99982,-0.680770
2,2,21,30,7,122,0.0,0,3316,549,25,...,0.35500,0.16465,0.32006,-0.045544,-0.31911,-1.01380,-0.054359,0.20074,0.32061,-0.455840
3,3,21,41,13,0,0.0,0,1670,293,11,...,-0.91080,0.69415,-0.32473,1.184100,0.45249,-1.03210,-1.265900,-0.74085,0.26441,-0.497070
4,4,9,29,13,33,1.0,0,7560,1211,67,...,-0.24938,1.02640,-0.10366,-0.379820,-0.82619,-0.20513,0.572080,0.43497,0.32462,-0.332930


In [46]:
cv = KFold(n_splits=5, shuffle=True, random_state=1234)
for fold_id, (train_index, valid_index) in enumerate(cv.split(all_new)):
    test_idx = valid_index
test_set = all_new[all_new.index.isin(test_idx)]
train_set = all_new[~(all_new.index.isin(test_idx))]

In [47]:
FEATURE_COLS = list(all_new.columns.drop(["id", "state","data_type"]))


TARGET_COLS = ["data_type"]
CATEGORICAL_COLS = [
    "country",
    "category1",
    "category2",
    "country+category1",
    "country+category2",
    "category1+category2",
]

params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
    }

for categorical_col in set(CATEGORICAL_COLS):
    if categorical_col not in FEATURE_COLS:
        CATEGORICAL_COLS.remove(categorical_col)
        print(f"{categorical_col} removed")

In [48]:
def run_optuna_lgbm(test, train, target_cols, feature_cols, categorical_cols):
    """
    import optuna.integration.lightgbm as lgb
    """
    X_train = train[feature_cols]
    y_train = train[target_cols]
    X_test = test[feature_cols]
    
    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))
    importances = []
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
        X_tr = X_train.iloc[train_index, :]
        X_val = X_train.iloc[valid_index, :]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[valid_index]

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=categorical_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=categorical_cols)
        
        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=False,
                          num_boost_round=1000,
                          early_stopping_rounds=5,
                          )
        
        oof_train[valid_index] = model.predict(X_val,
                                               num_iteration=model.best_iteration)
        y_pred = model.predict(X_test,
                               num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)
        
        
        # display importance
        importance = pd.DataFrame(model.feature_importance(), index=feature_cols, columns=['importance'])
        display(importance)
        importances.append(importance)

    return oof_train, sum(y_preds) / len(y_preds), importances

In [49]:
oof, preds, importances = run_optuna_lgbm(test_set, train_set, TARGET_COLS, FEATURE_COLS, CATEGORICAL_COLS)

[I 2021-01-25 11:44:16,317] A new study created in memory with name: no-name-244cfc71-2ed8-40d4-8ec8-8d0f8fa76764



  0%|          | 0/7 [00:00<?, ?it/s]


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/teppei/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Info] Number of positive: 6773, number of negative: 6724
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.259563 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 731072
[LightGBM] [Info] Number of data points in the train set: 13497, number of used features: 4721


/home/teppei/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/teppei/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501815 -> initscore=0.007261
[LightGBM] [Info] Start training from score 0.007261
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with




feature_fraction, val_score: 0.000000:   0%|          | 0/7 [00:11<?, ?it/s]


feature_fraction, val_score: 0.000000:  14%|#4        | 1/7 [00:11<01:10, 11.76s/it][I 2021-01-25 11:44:28,091] Trial 0 finished with value: 1.4757886134419414e-07 and parameters: {'feature_fraction': 0.5}. Best is trial 0 with value: 1.4757886134419414e-07.



feature_fraction, val_score: 0.000000:  14%|#4        | 1/7 [00:11<01:10, 11.76s/it]

...

bagging, val_score: 0.000000:  60%|######    | 6/10 [01:10<00:47, 11.90s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requireme

bagging, val_score: 0.000000:  70%|#######   | 7/10 [01:24<00:37, 12.42s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spl

KeyboardInterrupt: 